# Gravity Simulator
There are over a dozen constants that you can set to randomize and render the simulation. Objects follow Newton's gravitational formula.

In [1]:
import pygame
import random
import numpy as np

pygame 2.1.2 (SDL 2.0.18, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Constants

# Simulation settings
# if BALL_SIZES is defined, then NUM_BALLS, MIN_BALL_SIZE, and MAX_BALL_SIZE are ignored
BALL_SIZES = [250, 100, 80, 60, 50, *[random.randint(8, 15) for _ in range(20)]] # default None
NUM_BALLS = 4 # default 4
MIN_BALL_SIZE = 10 # default 10
MAX_BALL_SIZE = 100 # default 100
INITIAL_SPEED = 50 # default 20
GRAVITY_MULTIPLIER = 10 # default 25
# for each ball, only measure gravity from the k biggest ballsS
GRAVITY_FROM_TOP_K = 5 # default 20
COLLISIONS_ENABLED = False # default False
# how far over the edge of the screen a ball can go before it collides with the edge, set to None for no borders
BORDER_DEPTH = 300 # default 50
SEED = 1684488539 # default None

# Display settings
FPS = 120 # default 60
SCREEN_SIZE = (1600, 900, 1200) # default (1600, 900, 1200)
DEBUG = False # default False
SHADOWS_ENABLED = True # default True
# how much the apparent size of a ball changes as it moves away from the camera
SIZE_VARIATION = 1 # default 1 (min 0)
# how much the transparency of a ball changes as it moves away from the camera
ALPHA_VARIATION = 0.5 # default 1 (min 0, max 1)

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
SPACE = (15, 10, 50)

# Functions
def lerp(a, b, t):
    # clamp t
    t = max(0, min(1, t))
    return a + (b - a) * t

In [3]:
class Ball:
    def __init__(self, position, radius, color, screen_dims):
        self.position = position
        self.x = position[0]
        self.y = position[1]
        self.z = position[2]
        
        self.velocity = np.array(np.random.normal(0, INITIAL_SPEED, 3), dtype=float)
        self.gravity = np.zeros(3)

        self.radius = radius
        self.color = color
        self.mass = GRAVITY_MULTIPLIER * radius ** 3
        self.screen_dims = screen_dims

    def update_gravity(self, other):
        distance = np.linalg.norm(self.position - other.position)
        distance = max(distance, self.radius + other.radius)

        gravity_strength = other.mass / distance ** 2
        direction_unit = (other.position - self.position) / distance
        self.gravity += gravity_strength * direction_unit

    def update_gravity_from_all(self, balls):
        self.gravity = np.zeros(3)
        # calculate gravity from all balls
        for ball in balls:
            if ball is not self:
                self.update_gravity(ball)

    def update(self, delta_time):
        self.position += self.velocity * delta_time
        self.x = self.position[0]
        self.y = self.position[1]
        self.z = self.position[2]
        self.velocity += self.gravity * delta_time

        # check for collision with walls
        if BORDER_DEPTH is not None :
            for i in range(3):
                if self.position[i] < (self.radius - BORDER_DEPTH) \
                    and self.velocity[i] < 0:
                    self.velocity[i] = -self.velocity[i]
                if self.position[i] > (self.screen_dims[i] - self.radius + BORDER_DEPTH) \
                    and self.velocity[i] > 0:
                    self.velocity[i] = -self.velocity[i]

    def check_collision(self, other):
        distance = np.linalg.norm(self.position - other.position)
        r1, r2 = self.radius, other.radius
        p1, p2 = self.position, other.position
        v1, v2 = self.velocity, other.velocity

        return (distance < r1 + r2) and (p2 - p1).dot(v2 - v1) < 0
    
    def on_collision_none(self, other):
        pass
    
    def on_collision_advanced(self, other):
        self.velocity = -self.velocity
        r1, r2 = self.radius, other.radius
        m1, m2 = self.mass, other.mass
        p1, p2 = self.position, other.position
        v1, v2 = self.velocity, other.velocity
        
        # normal = (p2 - p1)/p1.distance(p2)
        # v1i = self.velocity.bounceVelocity(v1, 1, normal)
        # v2i = self.velocity.bounceVelocity(v2, 1, normal)
        
        # solve for final velocities (elastic collision)
        # equation 1: v1i + v1f = v2i + v2f
        # equation 2: m1v1i + m2v2i = m1v1f + m2v2f
        v1i, v2i = v1, v2
        
        # useful constants
        D = v2i - v1i
        P = m1 * v1i + m2 * v2i # sum(momentum)
        
        # final velocities
        v2f = (P - m1 * D)/(m1 + m2)
        v1f = v2f + D
        
        # normal direction, needed for bounciness calculation
        # normal = (p2 - p1)/p1.distance(p2)
        
        self.velocity = v1f
        other.velocity = v2f

    def draw(self, screen):
        # interpolate draw radius based on z position
        z_proportion = self.z / self.screen_dims[2]
        size_variation = max(SIZE_VARIATION, 0)
        # radius_multiplier = 2 / (3 * z_proportion + 1)
        radius_multiplier = (size_variation + 1) ** (1 - 2 * z_proportion)
        # clamp radius multiplier between 0.5 and 2
        radius_multiplier = min(max(radius_multiplier, 0.5), size_variation + 1)
        draw_radius = self.radius * radius_multiplier

        if SHADOWS_ENABLED:
            # draw shadow as though the light is coming from the top left
            shadow_radius = draw_radius * 1.15
            shadow = pygame.Surface((shadow_radius * 2, shadow_radius * 2), pygame.SRCALPHA)
            pygame.draw.circle(shadow, (0, 0, 0, 100), (shadow_radius, shadow_radius), shadow_radius)
            
            # find x and y distance to light
            light_location = (self.screen_dims[0] / 2, self.screen_dims[1] / 2)
            x_dist, y_dist = self.x - light_location[0], self.y - light_location[1]

            # draw with associated offset
            offset_distance = draw_radius * 0.15
            x_offset = offset_distance * x_dist ** 2 / (x_dist ** 2 + y_dist ** 2) * np.sign(x_dist)
            y_offset = offset_distance * y_dist ** 2 / (x_dist ** 2 + y_dist ** 2) * np.sign(y_dist)
            screen.blit(shadow, (self.x - draw_radius - (shadow_radius - draw_radius) + x_offset,
                                self.y - draw_radius - (shadow_radius - draw_radius) + y_offset))

        if ALPHA_VARIATION == 0:
            pygame.draw.circle(screen, self.color, (self.x, self.y), draw_radius)
        else:
            alpha_variation = min(max(ALPHA_VARIATION, 0), 1)
            min_alpha = lerp(255, 50, alpha_variation)
            ball_surface = pygame.Surface((draw_radius * 2, draw_radius * 2), pygame.SRCALPHA)
            pygame.draw.circle(ball_surface, (*self.color, lerp(min_alpha, 255, 1-z_proportion)),
                            (draw_radius, draw_radius), draw_radius)
            screen.blit(ball_surface, (self.x - draw_radius, self.y - draw_radius))

        if DEBUG:
            # set up font
            font = pygame.font.SysFont('Trebuchet MS', 18)
            # round position
            position_text = '|'.join(str(v) for v in np.round(self.position))
            # render text object
            text = font.render(position_text, True, (255, 255, 255))
            # draw text on ball
            screen.blit(text, (self.x - text.get_width() / 2, self.y - text.get_height() / 2))

class GravitySimulation:
    def __init__(self, screen_dims, ball_sizes):
        # randomly generate non-overlapping balls
        self.balls = []
        for ball_size in ball_sizes:
            while True:
                ball = Ball(
                    position = np.array([random.randint(0, screen_dims[0]),
                                         random.randint(0, screen_dims[1]),
                                         random.randint(0, screen_dims[2])], dtype=float),
                    radius = ball_size,
                    color = (random.randint(100, 240), random.randint(100, 240), random.randint(100, 240)),
                    screen_dims = screen_dims
                )
                if not any(ball.check_collision(other) for other in self.balls):
                    break
            self.balls.append(ball)
        
        self.top_k_balls = None

    def update(self, delta_time, top_k=None):
        if top_k is not None:
            if self.top_k_balls is None:
                self.top_k_balls = list(sorted(self.balls, key=lambda ball: ball.radius, reverse=True))[:top_k]
            balls = self.top_k_balls
        else:
            balls = self.balls

        for ball in self.balls:
            ball.update_gravity_from_all(balls)
            pass
        
        # check for collisions
        if COLLISIONS_ENABLED:
            for i in range(len(self.balls)):
                b1 = self.balls[i]
                for j in range(i + 1, len(self.balls)):
                    b2 = self.balls[j]
                    if b1.check_collision(b2):
                            b1.on_collision_advanced(b2)
                        
        for ball in self.balls:
            ball.update(delta_time)
    
    def draw(self, screen):
        # sort by z (high to low) so when we draw, the closer ones are on top
        self.balls.sort(key=lambda ball: ball.z, reverse=True)

        for ball in self.balls:
            ball.draw(screen)

In [4]:
if SEED is None:
    seed = random.randrange(0, 2**32 - 1)
    rng = random.Random(seed)
    print("Using random seed:", seed)
else:
    random.seed(SEED)

# Initialize the game engine
pygame.init()

# Set the height and width of the screen
screen = pygame.display.set_mode((SCREEN_SIZE[0], SCREEN_SIZE[1]))
pygame.display.set_caption("Gravity")

# Loop until the user clicks the close button.
done = False

# Used to manage how fast the screen updates
clock = pygame.time.Clock()

# Initialize simulation
ball_sizes = BALL_SIZES or [random.randint(MIN_BALL_SIZE, MAX_BALL_SIZE) for _ in range(NUM_BALLS)]
simulation = GravitySimulation(SCREEN_SIZE, ball_sizes)

# -------- Main Program Loop -----------
while not done:
    # --- Main event loop
    for event in pygame.event.get():  # User did something
        if event.type == pygame.QUIT:  # If user clicked close
            done = True  # Flag that we are done so we exit this loop

    # --- Game logic should go here
    simulation.update(1/FPS, top_k=GRAVITY_FROM_TOP_K)

    # First, clear the screen to white. Don't put other drawing commands
    # above this, or they will be erased with this command.
    screen.fill(SPACE)

    # --- Drawing code should go here
    simulation.draw(screen)

    # --- Go ahead and update the screen with what we've drawn.
    pygame.display.flip()

    # --- Set frames per second
    clock.tick(FPS)